# Refresh WFP data in Datamart

In [1]:
# Parameters to be injected
template_path = 'WFP_template.tsv'
datasets_path = './datasets'
datamart_api_url = 'http://localhost:12543'

In [3]:
import io
import os.path
import pandas as pd
import json
from requests import get,post,put,delete
from IPython.display import display, HTML

In [43]:
def upload_frame_annotated(buffer, url, put_data=True):
    
    buffer.seek(0)
    
    files = {
        'file': ('buffer.csv', buffer, 'application/octet-stream')
    }
    
    if put_data:
        response = put(url, files=files)
    else:
        response = post(url, files=files)
    if response.status_code == 400:
        print(json.dumps(response.json(), indent=2))
    else:
        print(json.dumps(response.json(), indent=2))

In [44]:
if not os.path.isfile(template_path):
    raise FileNotFoundError("Template file does not exist!")
    
df_template = pd.read_csv(template_path, sep='\t', dtype=object, header=None).fillna('')
df_template

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,dataset,WFP,WFP dataset,World food prices data compiled by the World F...,https://data.humdata.org/dataset/wfp-food-prices,,,,,,,,,,,,,
1,role,time,main subject,unit,qualifier,variable,unit,location,location,,qualifier,qualifier,qualifier,qualifier,qualifier,qualifier,qualifier,
2,type,%m/%d/%Y,string,string,string,number,string,country,admin1,,string,string,string,string,string,string,string,
3,description,,,,,,,,,,,,,,,,,
4,name,,,,,,,,,,,,,,,,,
5,unit,,,,,,,,,,,,,,,,,
6,header,date,cmname,unit,category,price,currency,country,admname,adm1id,mktname,mktid,cmid,ptid,umid,catid,sn,default


In [45]:
def generate_filenames(datasets_path: str):
    files_to_upload = []
    if os.path.isfile(datasets_path):
        files_to_upload.append(datasets_path)
    else:
        files_to_upload += [ os.path.join(datasets_path, f) for f in os.listdir(datasets_path) if os.path.isfile(os.path.join(datasets_path,f)) ]
    return files_to_upload

files_to_upload = generate_filenames(datasets_path)

In [46]:
dataset_id = df_template.iloc[0,1]
nCols = len(df_template.iloc[0]) - 1
for i, data_path in enumerate(files_to_upload):
        
    print(data_path)

    # Only extract the first nCols columns specified in the template
    sheet = pd.DataFrame()
    if data_path.endswith('.xlsx'):
        sheet = pd.read_excel(data_path, dtype=object).fillna('')
    elif data_path.endswith('.csv'):
        sheet = pd.read_csv(data_path, dtype=object).fillna('')
    else:
        continue
        
    sheet = sheet[sheet.columns[:nCols]]
    
    # Verify Label matches
    if False in sheet.columns == df_template.iloc[6][1:]:
        raise ValueError(f'Columns do not match between template and input: {data_path}. Abort...')
        
    # Build inputs
    sheet.insert(loc=0, column='', value='')
    sheet.iloc[0,0] = 'data'
            
    # Build annotated data
    sheet.columns = df_template.columns
    annotated_sheet = df_template.append(sheet)
        
    # post data to datamart
    buffer = io.StringIO()
    annotated_sheet.to_csv(buffer, index=False, header=False)
    url = f'{datamart_api_url}/datasets/{dataset_id}/annotated?create_if_not_exist=true'
    upload_frame_annotated(buffer, url, False)

./datasets\wfp_food_prices_ethiopia.csv
[
  {
    "name": "price",
    "variable_id": "price",
    "dataset_id": "WFP",
    "description": "variable column in QWFP",
    "corresponds_to_property": "PVARIABLE-QWFP-004",
    "qualifier": [
      {
        "name": "sn",
        "identifier": "PQUALIFIER-QWFP-011",
        "data_type": "String"
      },
      {
        "name": "catid",
        "identifier": "PQUALIFIER-QWFP-010",
        "data_type": "String"
      },
      {
        "name": "umid",
        "identifier": "PQUALIFIER-QWFP-009",
        "data_type": "String"
      },
      {
        "name": "ptid",
        "identifier": "PQUALIFIER-QWFP-008",
        "data_type": "String"
      },
      {
        "name": "cmid",
        "identifier": "PQUALIFIER-QWFP-007",
        "data_type": "String"
      },
      {
        "name": "mktid",
        "identifier": "PQUALIFIER-QWFP-006",
        "data_type": "String"
      },
      {
        "name": "mktname",
        "identifier": "PQUALIFI

### Check the data has been uploaded

In [5]:
dataset_id = 'WFP'
q_variable = 'price'
response = get(f'{datamart_api_url}/datasets/{dataset_id}/variables/{q_variable}')
df = pd.read_csv(io.StringIO(response.text)).head(20)
display(HTML(df.fillna('').to_html(index=False)))

dataset_id,variable_id,variable,main_subject,main_subject_id,value,value_unit,time,time_precision,country,admin1,admin2,admin3,region_coordinate,stated_in,stated_in_id,sn,catid,umid,ptid,cmid,mktid,mktname,category,located in the administrative territorial entity,stated in
WFP,price,price,cmname Avocados - Retail,QQWFP_cmname_Avocados___Retail,30.0,"KG, ETB",2020-01-15T00:00:00Z,,,,,,,,,3068_454_15_5,4,5,15,454,3068,Segno Gebeya,vegetables and fruits,Amhara Region,
WFP,price,price,cmname Avocados - Retail,QQWFP_cmname_Avocados___Retail,50.0,"KG, ETB",2020-04-15T00:00:00Z,,,,,,,,,497_454_15_5,4,5,15,454,497,Sekota,vegetables and fruits,Amhara Region,
WFP,price,price,cmname Avocados - Retail,QQWFP_cmname_Avocados___Retail,50.0,"KG, ETB",2020-05-15T00:00:00Z,,,,,,,,,497_454_15_5,4,5,15,454,497,Sekota,vegetables and fruits,Amhara Region,
WFP,price,price,cmname Avocados - Retail,QQWFP_cmname_Avocados___Retail,45.0,"KG, ETB",2020-06-15T00:00:00Z,,,,,,,,,3067_454_15_5,4,5,15,454,3067,Mersa,vegetables and fruits,Amhara Region,
WFP,price,price,cmname Avocados - Retail,QQWFP_cmname_Avocados___Retail,40.0,"KG, ETB",2020-06-15T00:00:00Z,,,,,,,,,475_454_15_5,4,5,15,454,475,Kobo,vegetables and fruits,Amhara Region,
WFP,price,price,cmname Avocados - Retail,QQWFP_cmname_Avocados___Retail,35.0,"KG, ETB",2020-08-15T00:00:00Z,,,,,,,,,3067_454_15_5,4,5,15,454,3067,Mersa,vegetables and fruits,Amhara Region,
WFP,price,price,cmname Avocados - Retail,QQWFP_cmname_Avocados___Retail,25.0,"KG, ETB",2020-08-15T00:00:00Z,,,,,,,,,3068_454_15_5,4,5,15,454,3068,Segno Gebeya,vegetables and fruits,Amhara Region,
WFP,price,price,cmname Bananas - Retail,QQWFP_cmname_Bananas___Retail,30.0,"KG, ETB",2020-01-15T00:00:00Z,,,,,,,,,497_254_15_5,4,5,15,254,497,Sekota,vegetables and fruits,Amhara Region,
WFP,price,price,cmname Bananas - Retail,QQWFP_cmname_Bananas___Retail,10.0,"KG, ETB",2020-01-15T00:00:00Z,,,,,,,,,3657_254_15_5,4,5,15,254,3657,Arba Minch,vegetables and fruits,"Southern Nations, Nationalities, and Peoples' Region",
WFP,price,price,cmname Bananas - Retail,QQWFP_cmname_Bananas___Retail,35.0,"KG, ETB",2020-01-15T00:00:00Z,,,,,,,,,3067_254_15_5,4,5,15,254,3067,Mersa,vegetables and fruits,Amhara Region,
